# Giveaway Entry Twitch App
Keeps track of when people redeem the `Giveaway Entry` channel point redeem

In [1]:
# for formatting
import json

# http requests
import requests

# generating secret keys
import secrets

# for improved parsing
from emoji import demojize

# load bot authorization data
from auth_data import bot_access_token, bot_client_id, user_access_token
'''
includes:
bot_access_token <- bot oAuth code
bot_client_id <- client id
user_access_token <- user access token, currently from Twitch CLI - https://dev.twitch.tv/docs/cli/
                     currently uses scopes
'''

# load username to channel id converter
from user2channelid import user2channelid
'''
usage:
    channel_ids = user2channelid(users_list, auth_token, client_id)
        input:
            user_list <- list of strings with Twitch usernames
            auth_token <- app auth token from a POST request from https://id.twitch.tv/oauth2/token
            client_id <- unique app client id
        
        output:
            channel_ids <- dict with {username : channel_id} key-value pairs
'''

import asyncio
# we use websockets because they have a nice python implementation, but webhook and conduits supposedly work with EventSub
from websockets.client import connect

We are using EventSub with Websockets to connect to the Twitch API. To do this, we need the EventSub URI.

In [2]:
# set the uri for eventsub
eventsub_uri = "wss://eventsub.wss.twitch.tv/ws"

Use `user2channelid` to get channel ids of the user and streamer

In [3]:
# load the usernames we want to use
from users_info import streamer, user

# get the ids
users_list = [streamer, user]
channel_ids = user2channelid(users_list, bot_access_token, bot_client_id)

# save them to variables
channel_id = channel_ids[streamer]
user_id = channel_ids[user]

# print output
# print(json.dumps(channel_ids, indent=3))

# pubsub topic subscriptions
# topics = [f"whispers.{user_id}"]
# topics = [f"channel-points-channel-v1.{channel_id}"]

In [ ]:
# currently unsure how to send a reconnect message to the server automatically
# using eventsub with websockets

all_chats = True
async def main():
    global eventsub_uri
    async with connect(eventsub_uri) as websockets:
        # get the welcome message and extract the session id
        welcome = await websockets.recv()
        welcome = json.loads(welcome)
        session_id = welcome["payload"]["session"]["id"]
        
        # subscribe to events with a POST request
        # https://dev.twitch.tv/docs/api/reference/#create-eventsub-subscription
        # says websockets requires a user access token - will fail if you use an app access token
        
        # subscribe to chat message event
        post_url = 'https://api.twitch.tv/helix/eventsub/subscriptions'
        post_headers = {"Authorization": "Bearer " + user_access_token,
                        "Client-Id" : bot_client_id,
                        "Content-Type" : "application/json"}
        # currently only able to subscribe to channel.chat.message because of the scopes we use.
        message_post_data = {
            "type" : "channel.chat.message",
            "version" : "1",
            "condition" : {
                "user_id" : str(user_id),
                "broadcaster_user_id" : str(channel_id)
            },
            "transport" : {
                "method" : "websocket",
                "session_id" : session_id
            }
        }
        message_req_post = requests.post(post_url, headers=post_headers, data=json.dumps(message_post_data))
        message_req_post = json.loads(message_req_post.text)
        if 'data' in message_req_post.keys():
            connect_date = message_req_post['data'][0]['created_at'][0:10]
            connect_time = message_req_post['data'][0]['created_at'][11:19]
            
            print(f"read chat status: {message_req_post['data'][0]['status']} on {connect_date} at {connect_time} UTC")
        else:
            print(f"read chat status: {message_req_post['status']}")
        # connect time
        
        # should be able to receive events now
        while True:
            # with EventSub, do not send any messages to the server or you will get a session disconnect
            # source: https://dev.twitch.tv/docs/eventsub/handling-websocket-events/
            # note that Pong messages are handled automatically by Python websockets
            result = await websockets.recv()
            result = json.loads(result)            

            if result['metadata']['message_type'] != 'session_keepalive':
                # make the json pretty for debugging
                res_nice = json.dumps(result, indent=3)
                try:
                    sub_type = result['metadata']['subscription_type']
                    if sub_type == 'channel.chat.message':
                        chat_date = result['metadata']['message_timestamp'][:10]
                        chat_time = result['metadata']['message_timestamp'][11:19]
                        chatter_user_name = result['payload']['event']['chatter_user_name']
                        chat_message = demojize(result['payload']['event']['message']['text'])
                        
                        if all_chats:
                            print(f"{chat_date} {chat_time} UTC | {chatter_user_name}: {chat_message}")
                        
                        # This is a workaround for now:
                        # When someone redeems "Giveaway Entry," 
                        if chatter_user_name == streamer and chat_message.startswith("Thank you for entering the monthly giveaway,"):
                            # the redeemer username comes after those words and a space but ends before an exclamation point
                            redeemer = chat_message[45:-2]
                            # print to stdout
                            if all_chats:
                                print("======================================")
                            print(f"{redeemer} redeemed monthly giveaway")
                            if all_chats:
                                print("======================================")
                                
                            # log it
                            # date {tab} time {tab} redeemer
                            # redeems.txt could also easily be a .csv, but either way it's getting imported into a google sheet for me
                            text_to_write = f"{chat_date}\t{chat_time}\t{redeemer}\n"
                            with open('redeems.txt', 'a') as redeem_file:
                                redeem_file.write(text_to_write)
                    else:
                        # print out the json response
                        print(res_nice)
                except:
                    print(res_nice)
                    if result['metadata']['message_type']=="session_reconnect":
                        with open('redeems.txt', 'a') as redeem_file:
                            redeem_file.write("reconnecting\n")
                        reconnect_url = result['payload']['reconnect_url']
                        eventsub_uri = reconnect_url
                        print("======================================")
                        print("RECONNECTING")
                        print("======================================")
                        break
        main()
await main()

read chat status: enabled on 2024-10-22 at 07:10:46 UTC
